In [5]:
import os
import pandas as pd
import time
import re
import tqdm
from ast import literal_eval
import pickle

In [6]:
PATH = '../data/'

DEV = True


if DEV: 
    f = 'DeepLearning'

In [7]:
with open(os.path.join(PATH, 'repos_as_pandas_metainfo_20230126.pkl'), 'rb') as file:
  df = pickle.load(file)

In [10]:
df = df[df.type_script=='py']

In [15]:
# CONVERT BYTES TO STRINGS 

df['code_script'] = df['code_script'].apply(lambda x: [line.decode() for line in x])


/var/folders/9b/jflbvjp576j7v4px00lmp6000000gn/T/ipykernel_94979/968818999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code_script'] = df['code_script'].apply(lambda x: [line.decode() for line in x])


In [55]:
df.meta_keywords

2        Bitcoin, Pricing Kernel, Bitcoin Option, Optio...
3        Bitcoin, Pricing Kernel, Bitcoin Option, Optio...
4        Bitcoin, Pricing Kernel, Bitcoin Option, Optio...
5        Bitcoin, Pricing Kernel, Bitcoin Option, Optio...
6        Bitcoin, Pricing Kernel, Bitcoin Option, Optio...
                               ...                        
19512    reinforcement learning, neural network, machin...
19514    reinforcement learning, neural network, machin...
19768                                                empty
19865    Shapley, additive model, SHAP, Interpretabilit...
19867    Shapley, Boosting, SHAP, Interpretability, Exp...
Name: meta_keywords, Length: 1912, dtype: object

In [56]:
df.columns

Index(['folder_name', 'metainfo_file', 'code_script', 'type_script',
       'script_name', 'q_name', 'meta_q_name', 'meta_desc', 'meta_keywords'],
      dtype='object')

In [58]:
df.meta_q_name.nunique()

658

In [61]:
df.metainfo_file.str.nunique()

AttributeError: 'StringMethods' object has no attribute 'nunique'

In [43]:
import ast
tree = ast.parse(df['code_script'].iloc[0][1])

In [49]:
f = ast.parse('def plot_MKM(\n')

SyntaxError: unexpected EOF while parsing (<unknown>, line 1)

In [44]:
for node in ast.walk(tree):
    print(node)
    print(node.__dict__)
    print("children: " + str([x for x in ast.iter_child_nodes(node)]) + "\\n")

{'body': [<ast.Import object at 0x7fb67ebd32e0>], 'type_ignores': []}
children: [<ast.Import object at 0x7fb67ebd32e0>]\n
{'names': [<ast.alias object at 0x7fb67ebd3550>], 'lineno': 1, 'col_offset': 0, 'end_lineno': 1, 'end_col_offset': 19}
children: [<ast.alias object at 0x7fb67ebd3550>]\n
{'name': 'pandas', 'asname': 'pd'}
children: []\n


In [42]:
node

In [45]:
node.__dict__

{'name': 'pandas', 'asname': 'pd'}

In [39]:
ast.dump(tree)

"Module(body=[Import(names=[alias(name='numpy', asname='np')])], type_ignores=[])"

In [52]:
%pip install astor
%pip install showast

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
import astor
import showast

In [27]:
astor.dump_tree(tree.body[0])

"Import(names=[alias(name='numpy', asname='np')])"

In [35]:
%%showast
1+2

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [17]:
df['code_script']

2        [import numpy as np\n, import pandas as pd\n, ...
3        [import pandas as pd\n, import os\n, from date...
4        [import dash\n, from dash.dependencies import ...
5                        [# required for QuantLet Graph\n]
6        [import numpy as np\n, \n, from spd_trading im...
                               ...                        
19512    [import pandas as pd\n, from datetime import d...
19514    [import pandas as pd\n, import numpy as np\n, ...
19768    [import matplotlib.pyplot as plt\r\n, import r...
19865    [#!/usr/bin/env python\n, # coding: utf-8\n, \...
19867    [#!/usr/bin/env python\n, # coding: utf-8\n, \...
Name: code_script, Length: 1912, dtype: object

In [5]:
def parse_imports(code):
    pattern_import = re.compile('(?<=import\s)\w+')
    matches_import = pattern_import.findall(code)

    pattern_from = re.compile('(?<=from\s)[\w|.]+(?=\simport)')
    matches_from = pattern_from.findall(df.Sourcecode[0])

    matches_import.extend(matches_from)
    return ' '.join(matches_import)


def parse_func_name(code):
    pattern = re.compile('(?<=def\s)[\w|.]+(?=\()')
    matches = pattern.findall(code)
    return ' '.join(matches)


def parse_func_args(code, one_letter_removal=True):
    pattern = re.compile('(?<=def).+(?=\))')
    matches = pattern.findall(code)
    final_matches = ''
    for match in matches:
        final_matches += match.split('(')[1]
    final_matches = final_matches.replace(',', ' ')

    if one_letter_removal:
        new_matches = []
        for word in final_matches.split(' '):
            if len(word)>1:
                new_matches.append(word)

        final_matches = ' '.join(new_matches)

    #print(new_matches)

    return final_matches

def parse_vars(code):
    pass

def parse_all(code):
    pattern = re.compile('(?<=import\s)\w+|(?<=from\s)[\w|.]+(?=\simport)|(?<=def\s).+(?=\))|(?<=#).+')
    matches = pattern.findall(code)
    new_matches = []
    for match in matches:
        if '(' in match:
            new_matches.extend(match.split('('))
        else:
            new_matches.append(match)
        
    return ' '.join(new_matches).replace(',', ' ')

def parse_comments(code):
    pattern = re.compile('(?<=#).+')#|(?<=\"\"\")(?s).+(?=\"\"\")
    matches = pattern.findall(code)
    return matches

def parse_docstrings(code):
    pattern = re.compile('(?<=\"\"\")(?s).+?(?=\"\"\")')
    matches = pattern.findall(code)
    return matches
    


In [6]:
df['ParsedSource'] = df.Sourcecode.apply(parse_all)

In [7]:
df['num_parsed_code_tokens'] = df['ParsedSource'].apply(lambda x: len(x.split()))

In [9]:
del df['Sourcecode']

In [10]:
df.to_csv('../data/parsed_source_code_df_v2.csv', index=False)

In [ ]:
# TODOS:

# remove ------- from comments
# remove ##### from comments
# remove one letter variables
# remove 
# split Camel Cases
# split on _ 
# split on .

In [ ]:
##### ENCODE THE DESCRIPTION

In [ ]:
##### ENCODE THE CODE

In [ ]:
##### ENCODE THE KEYWORDS